In [1]:
##Importing packages we need##

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation
from matplotlib import cm
from matplotlib.cm import get_cmap
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import metpy.calc as mpcalc
from metpy.units import units
from numpy import *
import xarray as xr
from netCDF4 import Dataset, num2date
import math
import pygrib
import cdsapi
import imageio
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import metpy as mp
import warnings
import glob
import dask
warnings.filterwarnings("ignore")

print("Done importing modules")

Done importing modules


In [2]:
##Data Processing Function##

def process_CF_data(ds_sfc):
    '''This function does a lot of calculations and outputs the timeseries with CAA and FLF.'''
    
    ##Load in the datasets and read in variables and cords##

    #This is for the surface level#
    dew_2m = ds_sfc.d2m.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    uwnd_10m = ds_sfc.u10.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    vwnd_10m = ds_sfc.v10.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    tmpk_2m = ds_sfc.t2m.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    mslp = ds_sfc.msl.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    sfc_p = ds_sfc.sp.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))

    #Extract Coordinates#
    lats = ds_sfc.latitude.metpy.sel(latitude=slice(latN,latS))
    lons = ds_sfc.longitude.metpy.sel(longitude=slice(lonW,lonE))
    lons_2D, lats_2D = meshgrid(lons,lats)
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

    tmpk_2m
    
    ##Apply a spatial smoother to the variables so that synoptic-scale signals can be more readily observed (25)##

    smoothing_var = 15

    #Surface variables#
    dew_2m = mpcalc.smooth_gaussian(dew_2m, smoothing_var)
    uwnd_10m = mpcalc.smooth_gaussian(uwnd_10m, smoothing_var)
    vwnd_10m = mpcalc.smooth_gaussian(vwnd_10m, smoothing_var)
    tmpk_2m = mpcalc.smooth_gaussian(tmpk_2m, smoothing_var)
    mslp = mpcalc.smooth_gaussian(mslp, smoothing_var)
    sfc_p = mpcalc.smooth_gaussian(sfc_p, smoothing_var)
    
    ##Calculate additional variables to plot##

    #Surface level variables#
    thetaE = mpcalc.equivalent_potential_temperature(sfc_p, tmpk_2m, dew_2m) 
    thetaE_adv = mpcalc.advection(thetaE, uwnd_10m, vwnd_10m)

    #Scale by a factor#
    thetaE_adv_s = thetaE_adv * 1e4

    #Select only CAA and WAA#
    thetaE_adv_s_CAA = thetaE_adv_s.where(thetaE_adv_s < 0, other=np.nan)
    thetaE_adv_s_WAA = thetaE_adv_s.where(thetaE_adv_s > 0, other=np.nan)

    thetaE_adv_s
    
    ##Calculate Front Locator Function (FLF) using ThetaE##

    def calculate_flf(thetaE, dx, dy):
        '''Make an FLF function to calculate it for all gridpoints at each timestep.'''

        #Step 1: Calculate the gradient of thetaE#
        d0_dy, d0_dx = mpcalc.gradient(thetaE, deltas=(dy, dx)) 

        #Step 2: Calculate the magnitude of the gradient#
        mag_grad_thetaE = np.sqrt((d0_dx**2) + (d0_dy**2))  # |grad(thetaE)|

        #Step 3a: Calculate the gradient of the magnitude#
        grad_step2_y, grad_step2_x = mpcalc.gradient(mag_grad_thetaE, deltas=(dy, dx))

        #Step 3b: Redo but for Qn component only#
        qx = grad_step2_x.copy()
        qy = grad_step2_y.copy()
        denominator = ((d0_dx) * (d0_dx)) + ((d0_dy) * (d0_dy))

        Qn_x = (((qx) * (d0_dx) * (d0_dx)) + ((qy) * (d0_dy) * (d0_dx))) / (denominator)  #i component
        Qn_y = (((qx) * (d0_dx) * (d0_dy)) + ((qy) * (d0_dy) * (d0_dy))) / (denominator)  #j component
        #Qn_low = Qn_x + Qn_y
        #Qn = Qn_low * 1e10

        #Step 4: Calculate the divergence of the gradient of the magnitude with Qn component#
        div_step4_Qn = mpcalc.divergence(Qn_x, Qn_y, dx=dx, dy=dy)
        div_step4_Qn_convert = div_step4_Qn * 1e14  #What we need

        return div_step4_Qn_convert

    #Apply the function across all timesteps now#
    FLF_data = xr.apply_ufunc(
        calculate_flf, 
        thetaE, 
        input_core_dims=[['latitude', 'longitude']],  #Core dimensions over which to apply the function
        output_core_dims=[['latitude', 'longitude']], #Core dimensions on the output
        vectorize=True,  #Automatically vectorize the computation if needed
        dask='parallelized',  #Use Dask for parallel computation if thetaE is a Dask-backed array
        kwargs={'dx': dx, 'dy': dy})  #Additional keyword arguments for the function
    
    ##See the CF dataset and some days to select##
    
    ds_CF = xr.open_dataset('CF_xr.nc').metpy.parse_cf()
    
    ##The grid cell ranking method##

    #Remake the spatial heatmap#
    cold_front_counts = ds_CF['CF_Dataset'].sum(dim='time')

    #Sort the counts to obtain ranks for each grid cell#
    flattened_counts = cold_front_counts.values.flatten()
    sorted_indices = np.argsort(-flattened_counts)  #Negative for descending sort

    ranks = np.empty_like(sorted_indices)
    ranks[sorted_indices] = np.arange(len(sorted_indices)) + 1  #Rank starts from 1

    #Plot the heatmap plot underneath#
    fig, ax = plt.subplots(figsize=(14, 10))
    heatmap = ax.imshow(cold_front_counts, cmap='viridis', origin='lower') 

    plt.colorbar(heatmap, label='Number of Cold Fronts')
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)
    ax.set_title('Cold Front Occurrences Heatmap with Ranked Grid Cells', fontsize=16)

    #Fix the lons and lats#
    lat_vals = np.flip(ds_CF['latitude'].values)  
    lon_vals = ds_CF['longitude'].values  

    ax.set_xticks(np.arange(len(lon_vals)))
    ax.set_yticks(np.arange(len(lat_vals)))
    ax.set_xticklabels([f'{lon:.2f}' for lon in lon_vals])
    ax.set_yticklabels([f'{lat:.2f}' for lat in lat_vals])
    plt.xticks(rotation=45)

    #Convert ranks to 2D grid and print them out on cells#
    ranks_2d = ranks.reshape(cold_front_counts.shape)
    lon = cold_front_counts['longitude'].values
    lat = cold_front_counts['latitude'].values

    #Overlay original smallest rectangle#
    rect_domain2 = patches.Rectangle(

        (5-0.5, 5-0.5), #(lon_min, lat_min)
        13 - 5,  #width (lon_max - lon_min)
        11 - 5,  #height (lat_max - lat_min)
        linewidth=4,
        edgecolor='black',
        facecolor='none'
    )

    ax.add_patch(rect_domain2)

    #Print out the ranks in each grid cell#
    for i, row in enumerate(ranks_2d):   
            for j, rank in enumerate(row):

                if rank <= 238: 
                    ax.text(j, i, str(rank), ha='center', va='center', color='black', fontsize=10)

    #Now make a mask for the highest frequency 52 cells#
    top_cells = np.dstack(np.unravel_index(np.argsort(ranks_2d.ravel()), ranks_2d.shape))[0][:52]

    #Initialize the binary mask array#
    binary_mask_tc = np.zeros_like(ranks_2d, dtype=bool)

    #Set the top 52 cells in the mask to True#
    for cell in top_cells:
        binary_mask_tc[cell[0], cell[1]] = True

    #Add outlines for the top 52 cells#
    for cell in top_cells:
        rect = patches.Rectangle((cell[1]-0.5, cell[0]-0.5), 1, 1, linewidth=2, edgecolor='purple', facecolor='none')
        ax.add_patch(rect)

    plt.show()
    
    ##We want to extract the FLF values for the Eastern CO domain##

    #First remove all the frontolysis values#
    FLF_data_FG = FLF_data.where(FLF_data <= 0, other=np.nan)

    #Obtain the CAA for Theta#
    thetaE_CAA = thetaE_adv_s_CAA.copy()

    #Define extents and subset#
    ECO_area = [41, -105.25, 37, -102]  #Eastern CO  105.25

    FLF_ECO = FLF_data_FG.sel(latitude=slice(ECO_area[0],ECO_area[2]),longitude=slice(ECO_area[1],ECO_area[3]))
    thetaE_CAA_ECO = thetaE_CAA.sel(latitude=slice(ECO_area[0],ECO_area[2]),longitude=slice(ECO_area[1],ECO_area[3]))

    #Define a start date#
    start_date = str(FLF_ECO['time'][0].values)

    #Create a second permanent mask using the 52 highest freq. gridcells and subset data#
    binary_mask_tcf = np.flipud(binary_mask_tc)

    FLF_top52flag = FLF_ECO * binary_mask_tcf
    thetaE_CAA_top52flag = thetaE_CAA_ECO * binary_mask_tcf

    #Accumulate the FLF, ThetaEadv for the different regions#
    FLF_accumulated_top52 = abs(FLF_top52flag).sum(dim=['latitude', 'longitude'])
    thetaE_CAA_accumulated_top52 = abs(thetaE_CAA_top52flag).sum(dim=['latitude', 'longitude'])

    #Plotting with pandas dates#
    dates = pd.date_range(start=start_date, periods=FLF_accumulated_top52.sizes['time'], freq='H')
    dates = pd.to_datetime(dates)

    FLF_timeseries = pd.Series(data=FLF_accumulated_top52.values, index=dates)
    FLF_timeseries2 = pd.Series(data=thetaE_CAA_accumulated_top52.values, index=dates)

     #Try an overlap with FLF and CAA now#

    #Create a new DataArray that only contains FLF values where CAA is non-zero#
    FLF_and_CAA = xr.where(thetaE_CAA_top52flag.values < 0, FLF_top52flag, 0)  
    FLF_CAA_accumulated = abs(FLF_and_CAA).sum(dim=['latitude', 'longitude'])
    FLF_CAA_timeseries = pd.Series(data=FLF_CAA_accumulated.values, index=dates)

    FLF_timeseries_conv = FLF_CAA_timeseries * FLF_timeseries2   #combine orange and blue lines

     #Plotting#

    #Make plot 1 for T52#
    fig, ax = plt.subplots(figsize=(10,8))
    ax2 = ax.twinx() 
    ax.plot(FLF_timeseries.index, FLF_timeseries.values, linewidth=3, color='black', label='FLF T52')
    ax2.plot(FLF_timeseries2.index, FLF_timeseries2.values, linewidth=3, color='blue', label='CAA T52')
    ax.plot(FLF_CAA_timeseries.index, FLF_CAA_timeseries.values, linewidth=3, color='orange', label='T52: FLF and CAA')
    ax2.plot(FLF_timeseries_conv.index, FLF_timeseries_conv.values, linewidth=3, color='purple', linestyle='--', label='Weighted')

    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.xticks(rotation=45)
    ax.set_xlabel('Time', fontsize=14)
    ax.set_ylabel('Accumulated Scaled FLF', fontsize=14)
    ax2.set_ylabel('Accumulated Scaled CAA', fontsize=14)  #Label for the second y-axis
    ax.set_title(f'Timeseries of Accumulated FLF,CAA T52 Method for {dates[47]}', fontsize=16)  #change
    ax.set_xlim([dates[12],dates[60]])
    ax.grid(True)  
    plt.tight_layout() 
    plt.savefig(f'FLF_Mult_{dates[47]}.png')
    plt.show()
    
    #Make plot 2 for T52#
    fig, ax = plt.subplots(figsize=(10,8))
    ax2 = ax.twinx() 
    ax.plot(FLF_timeseries.index, FLF_timeseries.values, linewidth=3, color='black', label='FLF T52')
    ax2.plot(FLF_timeseries2.index, FLF_timeseries2.values, linewidth=3, color='blue', label='CAA T52')
    ax.plot(FLF_CAA_timeseries.index, FLF_CAA_timeseries.values, linewidth=3, color='orange', label='T52: FLF and CAA')
    ax.plot(FLF_timeseries_conv.index, FLF_timeseries_conv.values, linewidth=3, color='purple', linestyle='--', label='Weighted')

    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.xticks(rotation=45)
    ax.set_xlabel('Time', fontsize=14)
    ax.set_ylabel('Accumulated Scaled FLF', fontsize=14)
    ax2.set_ylabel('Accumulated Scaled CAA', fontsize=14)  #Label for the second y-axis
    ax.set_title(f'Timeseries of Accumulated FLF,CAA T52 Method for {dates[47]}', fontsize=16)  #change
    ax.set_xlim([dates[12],dates[60]])
    ax.grid(True)  
    plt.tight_layout() 
    plt.savefig(f'CAA_Mult_{dates[47]}.png')
    plt.show()
    
    return FLF_timeseries_conv

In [3]:
# ##Declare time, level, lat/lon boundaries and loop through a 2-day period for Frontal Analysis##

#Area and Time#
colorado_area = [44, -112, 34, -99]  #some buffer on all sides
latN = 44
latS = 34
lonW = -112    #Must be in degrees E (The western hemisphere is captured between 180 and 360 degrees east)
lonE = -99
level = 850    #What level do we want to look at FLF on

In [4]:
##Load in the Org and Manual tables and perform a replacement##

# #Load the 2 datasets@
# df_CF_MANMF = pd.read_csv('CF_Cases _ManF_MF.csv')
# df_CF_MANMF

# ds_CF_mf = pd.read_csv('Group_Tables/csv/CF_mf.csv')
# ds_CF_mf

# Load the original and manually corrected tables
df_original = pd.read_csv('Group_Tables/csv/CF_unclear.csv')
df_corrected = pd.read_csv('CF_Cases_ManF_UNC.csv')

# Ensure date columns are in the same format
df_original['date'] = pd.to_datetime(df_original['date'])
df_corrected['Date'] = pd.to_datetime(df_corrected['Date'])

# Display the original and corrected DataFrames
print("Original DataFrame:")
print(df_original.head())

print("\nCorrected DataFrame:")
print(df_corrected.head())

# Merge the original and corrected DataFrames
df_updated = df_original.merge(df_corrected[['Date', 'Max Time']], left_on='date', right_on='Date', how='left')

# Update the original DataFrame with the corrected max_time values
df_updated['max_time'] = df_updated['Max Time'].combine_first(df_updated['max_time'])

# Format the max_time column to ensure zero-padded hours
df_updated['max_time'] = pd.to_datetime(df_updated['max_time']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Drop the extra 'Date' and 'Max Time' columns used for merging
df_updated = df_updated.drop(columns=['Date', 'Max Time'])

# Save the updated DataFrame to a new CSV file or overwrite the original one
df_updated.to_csv('Group_Tables/csv/CF_unclear_updated.csv', index=False)

print("\nUpdated DataFrame:")
print(df_updated.head())

Original DataFrame:
        date    max_value             max_time
0 1959-01-04  3741.203221  1959-01-02 12:00:00
1 1990-01-04   160.218851  1990-01-03 18:00:00
2 1989-01-08   580.408035  1989-01-07 10:00:00
3 2007-01-13  1658.155443  2007-01-11 21:00:00
4 1967-01-15   370.152775  1967-01-14 17:00:00

Corrected DataFrame:
        Date             Max Time
0 2059-01-04  1959-01-02 12:00:00
1 2021-01-15   2021-01-14 8:00:00
2 1975-01-22  1975-01-21 14:00:00
3 1993-01-24  1993-01-23 11:00:00
4 1989-02-02  1989-02-01 14:00:00

Updated DataFrame:
        date    max_value             max_time
0 1959-01-04  3741.203221  1959-01-02 12:00:00
1 1990-01-04   160.218851  1990-01-03 18:00:00
2 1989-01-08   580.408035  1989-01-07 10:00:00
3 2007-01-13  1658.155443  2007-01-11 21:00:00
4 1967-01-15   370.152775  1967-01-14 17:00:00


In [ ]:
# ##Import the CF date groups and read the data in with the correct year##

# #Load the CSV data into a pandas df#
# CF_groups = pd.read_csv('CF_Groups.csv')

# #Retain dates between January 1, 1950 and December 31, 2022#
# def refine_year(date):
    
#     if date.year < 1950:
#         return date + pd.DateOffset(years=100)  
    
#     elif date.year > 2022:
#         return date - pd.DateOffset(years=100)  
    
#     return date

# #Extract the columns and convert to datetime objects#
# CF_groups['Good'] = pd.to_datetime(CF_groups['Good'], format='%m/%d/%y', errors='coerce').apply(refine_year)
# CF_groups['Multi Front'] = pd.to_datetime(CF_groups['Multi Front'], format='%m/%d/%y', errors='coerce').apply(refine_year)
# CF_groups['Unclear '] = pd.to_datetime(CF_groups['Unclear '], format='%m/%d/%y', errors='coerce').apply(refine_year)

# #Convert to xr#
# CF_groups_xr = CF_groups.to_xarray()
# CF_groups_xr

In [ ]:
##Perform some data munging##

#Read in the dataset#
# df_CF_MANMF = pd.read_csv('CF_Cases _ManF_MF.csv')
# df_CF_MANMF

# #Shift the index to start from 1 instead of 0#
# df_CF_good.index = df_CF_good.index + 1
# df_CF_good

#Read in the dataset#
#df_CF_mf = pd.read_csv('CF_multifront_org.csv')
# df_CF_mf

# #Shift the index to start from 1 instead of 0#
# df_CF_mf.index = df_CF_mf.index + 1
# df_CF_mf[25:35]

# #Read in the dataset#
# df_CF_unclear = pd.read_csv('CF_unclear_org.csv')
# df_CF_unclear

# #Shift the index to start from 1 instead of 0#
# df_CF_unclear.index = df_CF_unclear.index + 1
# df_CF_unclear[80:90]

# #####

# #Drop some selected rows#
# indices_to_delete = [88]
# df_CF_unclear_nodup = df_CF_unclear.drop(indices_to_delete)
# df_CF_unclear_nodup

# #Reset the index#
# df_CF_unclear_nodupr = df_CF_unclear_nodup.reset_index(drop=True)
# df_CF_unclear_nodupr.index = df_CF_unclear_nodupr.index + 1

# #save#
# df_CF_unclear_nodupr.to_csv('CF_unclear.csv', index=False)
#df_CF_unclear_nodupr

In [ ]:
##Now plot the distribution of max values and a frequency of CF based on passage time##

#Extract hour from 'max_time' and create a new column#
df_CF_unclear_nodupr['hour'] = pd.to_datetime(df_CF_unclear_nodupr['max_time']).dt.hour

#Count the frequency of each hour#
hour_counts = df_CF_unclear_nodupr['hour'].value_counts().sort_index()

#Plot the frequency of cold fronts by hour#
plt.figure(figsize=(14, 6))
plt.bar(hour_counts.index, hour_counts.values, color='skyblue', edgecolor='black')
plt.title('Frequency of "Unclear" Cold Fronts by Hour', fontsize=18)
plt.xlabel('UTC Time', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.xticks(range(24))
plt.grid(axis='y')
plt.text(0.98, 0.95, 'N=333', horizontalalignment='right', verticalalignment='top', transform=plt.gca().transAxes, fontsize=14)
plt.show()

# Plot the distribution of max values
plt.figure(figsize=(14, 6))
plt.hist(df_CF_unclear_nodupr['max_value'], bins=30, color='orange', edgecolor='black')
plt.title('Distribution of Max Values Unclear', fontsize=18)
plt.xlabel('Max Value', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y')
plt.text(0.98, 0.95, 'N=333', horizontalalignment='right', verticalalignment='top', transform=plt.gca().transAxes, fontsize=14)
plt.show()

In [ ]:
# ##Find the duplicates##

# # #Round and Check for duplicates#
# df_CF_good_r = df_CF_good.copy()
# df_CF_good_r['max_value'] = df_CF_good_r['max_value'].round()

# duplicates = df_CF_good_r[df_CF_good_r.duplicated('max_value', keep=False)]
# duplicates

In [ ]:
# df_CF_unclear_nodupr

In [ ]:
##Now lets manually fix some dates and cases##

# row = None

# #Change the max time and value#
# df_CF_good.at[row, 'max_time'] = '2022-01-01 00:00:00'
# df_CF_good.at[row, 'max_value'] = 1234.56


In [ ]:
##Now put everything under a loop##

#Use the "good" dates#
results_unclear = []

for date in CF_groups_xr['Unclear '].values:
    
    py_date = pd.to_datetime(date).to_pydatetime()
    formatted_date = py_date.strftime('%m_%d_%Y')   #This will format the date as 'MM_DD_YYYY'
    file_path = f"CF_DATA/ERA5_{formatted_date}.nc"
    
    try:
        #Read in the data#
        ds_sfc = xr.open_dataset(file_path).metpy.parse_cf()
        
        #Apply the function#
        FLF_timeseries_conv = process_CF_data(ds_sfc)  
        
        #Locate the maximum value and time#
        max_index = FLF_timeseries_conv[12:60].idxmax()
        max_value = FLF_timeseries_conv[max_index]
        
        #Store the results#
        results_unclear.append({'date': date, 'max_value': max_value, 'max_time': max_index})
        print(f"The maximum value is {max_value} and it occurs on {max_index}")
    
    except FileNotFoundError:
        
        print(f"File not found for date: {formatted_date}")
        
        continue
        
#Convert results to DataFrame or xarray Dataset#
results_df_unclear = pd.DataFrame(results_unclear)

In [ ]:
# ##Convert to netcdf#

# df_CF_mf_nodupr
# df_CF_mf_nodupr.to_csv('CF_unclear.csv', index=False)

# results_ds_unclear = results_df_unclear.set_index('date').to_xarray()
# netcdf_path = 'CF_t0_unclear'
# results_ds_unclear.to_netcdf(netcdf_path)